# HW1: zhsegment

In [1]:
from zhsegment import *

In [2]:
class Entry:
    def __init__(self, word, start_pos, log_prob, back_ptr):
        self.word = word
        self.start_pos = start_pos
        self.log_prob = log_prob
        self.back_ptr = back_ptr

In [3]:
# sorting function
def sortbyprob(entry):
    "Function for sorting by probability"
    return entry.log_prob

## 1. First attempt: baseline (unigram)

In [4]:
class Segment:

    def __init__(self, Pu):
        self.Pu = Pu

    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []

        # initialize the chart
        chart = [ None for i in range(len(text))]
        heap = []
 
        # initialize the heap
        # push words start from position 0
        for i in range(len(text)):
            word = text[: (i + 1)]
            if word in self.Pu.keys() :
                entry = Entry(word, 0, log10(self.Pu(word)), None)
                heap.append(entry)

        # iteratively fill in chart[i] for i
        while heap:
            # sort in probability descending order and pop from heap
            heap.sort(key=sortbyprob, reverse=True)
            entry = heap.pop(0)
            endindex = entry.start_pos + len(entry.word) - 1
            # compute argmax
            if chart[endindex] is not None:
                preventry = chart[endindex]
                if entry.log_prob > preventry.log_prob:
                    chart[endindex] = entry
                else:
                    continue 
            else:
                chart[endindex] = entry
            # push new entry to the heap (starting from (endindex + 1))
            for i in range((endindex + 1), len(text)):
                newword = text[(endindex + 1): (i + 1)]
                if newword in self.Pu.keys() :
                    newentry = Entry(newword, endindex + 1, entry.log_prob + log10(self.Pu(newword)), entry)
                    if newentry not in heap:
                        heap.append(newentry)

        # get the best segmentation
        segmentation = []
        finalindex = len(text) - 1
        finalentry = chart[finalindex]
        while finalentry != None:
            segmentation.append(finalentry.word)
            finalentry = finalentry.back_ptr

        segmentation.reverse()

        return segmentation

In [5]:
Pu = Pdist(data=datafile("../data/count_1w.txt"))
segmenter = Segment(Pu) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签订 高 科技 合作 协议

“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


### Evaluate dev.out

In [6]:
import sys
sys.path.append("..")
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)


score: 0.48


### Discussion

* The first attempt calculuates the probability based on P(w1,....wn) = c(w1,....,wn)/N. The problem with this is that it does not generalize well to new unseen words from the dictonary, which we also need to predict the most likely segmentation of. The bigram model will be used in the second attempt to predict for which words are more likely to get paired together with the unseen words. Unseen words in attempt 1 will not get pushed into the heap since it is not part of the dictionary, so we won't be able to compute the probability for it.

* Also for the first attempt, dividing by N can cause issues where long words have greater probability than shorter words, even though shorter words are more likely to occur, so the avoid_long_words function from HW0 (for scaling down longer segments) will be applied in the second attempt as well.

* Now, we scored only 0.48 in our attempt 1. Also, one sentence did not output its segmentation result because of unseen words. To deal with new words that are not in the dictionary, we are going to push unseen words with length 1 into the heap with a probability calculated by the missing function.

## 2. Second attempt: baseline (bigram)

In [7]:
def avoid_long_words(key, N):
    log_prob = log10(10.) - log10(N * 10000 ** len(key)) #equivalent to log(10./(N * 10000 ** len(key))) , used to avoid arithmentic underflow for small probabilities
    return 10**log_prob #convert log back to its probability

In [8]:
class Segment:

    def __init__(self, Pu, Pb, lam):
        self.Pu = Pu
        self.Pb = Pb
        self.lam = lam # lambda for Jelinek-Mercer Smoothing

    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []

        # initialize the chart
        chart = [ None for i in range(len(text))]
        heap = []
 
        # initialize the heap
        # push words start from position 0
        for i in range(len(text)):
            word = text[: (i + 1)]
            if word in self.Pu.keys() or len(word) == 1:
                entry = Entry(word, 0, log10(self.Pu(word)), None)
                heap.append(entry)

        # iteratively fill in chart[i] for i
        while heap:
            # sort in probability descending order and pop from heap
            heap.sort(key=sortbyprob, reverse=True)
            entry = heap.pop(0)
            endindex = entry.start_pos + len(entry.word) - 1
            # compute argmax
            if chart[endindex] is not None:
                preventry = chart[endindex]
                if entry.log_prob > preventry.log_prob:
                    chart[endindex] = entry
                else:
                    continue 
            else:
                chart[endindex] = entry
            # push new entry to the heap (starting from (endindex + 1))
            for i in range((endindex + 1), len(text)):
                newword = text[(endindex + 1): (i + 1)]
                bigram = entry.word + " " + newword
                if newword in self.Pu.keys() or len(newword) == 1:
                    # compute conditional probability (cpr)
                    if bigram in self.Pb.keys() and entry.word in self.Pu.keys():
                        cpr = self.Pb[bigram] / self.Pu[entry.word]
                    else:
                        cpr = self.Pu(newword)
                    # apply Jelinek-Mercer Smoothing
                    newentry = Entry(newword, endindex + 1, entry.log_prob + log10(self.lam * cpr + (1 - self.lam) * self.Pu(newword)), entry)
                    if newentry not in heap:
                        heap.append(newentry)

        # get the best segmentation
        segmentation = []
        finalindex = len(text) - 1
        finalentry = chart[finalindex]
        while finalentry != None:
            segmentation.append(finalentry.word)
            finalentry = finalentry.back_ptr

        segmentation.reverse()

        return segmentation

In [9]:
Pu = Pdist(data=datafile("../data/count_1w.txt"), missingfn=avoid_long_words)
Pb = Pdist(data=datafile("../data/count_2w.txt"), missingfn=lambda x, y: 0)
lam = 0.2
segmenter = Segment(Pu, Pb, lam) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


In [10]:
import sys
sys.path.append("..")
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

score: 0.87


### Discussion

* As you can see, there are some sentences in attempt one without output. The reason is that there are some unseen words not in the keys of built dictionary, which led to the unseen words will not be pushed into the heap. In this way, it would quit algorithm during the process. What we did is to push the unseen words with length one into the heap even though it’s not in the dictionary. 
* Besides, we added missingfn to Pdist class. The code is from our HW0.
* Another thing we improved here is we added bigram method here and implemented Jelinek-Mercer Smoothing. If the counts of unigram and bigram both exist, we used cpr = self.Pb[bigram] / self.Pu[entry.word] to calculate maximum likelihood estimation (MLE). If not, we used cpr = self.Pu(newword) to calculate MLE. After that, we used self.lam * cpr + (1 - self.lam) * self.Pu(newword) to calculate smoothing probability.

From the output we can see that, the missing sentence is shown up now. And we scored 0.87 in dev.txt.

## 3. Third attempt: tuning parameters (lambda, maxlen)

In [11]:
class Segment:

    def __init__(self, Pu, Pb, lam):
        self.Pu = Pu
        self.Pb = Pb
        self.lam = lam   # lambda for Jelinek-Mercer Smoothing

    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []

        # initialize the chart
        chart = [ None for i in range(len(text))]
        heap = []
        maxlen = 10 # max length of words (avoid too long words)

        # initialize the heap
        # push words start from position 0
        for i in range(len(text)):
            if i == maxlen:
                break
            word = text[: (i + 1)]
            if word in self.Pu.keys() or len(word) <= 3:
                entry = Entry(word, 0, log10(self.Pu(word)), None)
                heap.append(entry)

        # iteratively fill in chart[i] for i
        while heap:
            # sort in probability descending order and pop from heap
            heap.sort(key=sortbyprob, reverse=True)
            entry = heap.pop(0)
            endindex = entry.start_pos + len(entry.word) - 1
            # compute argmax
            if chart[endindex] is not None:
                preventry = chart[endindex]
                if entry.log_prob > preventry.log_prob:
                    chart[endindex] = entry
                else:
                    continue 
            else:
                chart[endindex] = entry
            # push new entry to the heap (starting from (endindex + 1))
            for i in range((endindex + 1), len(text)):
                if i - endindex - 1 == maxlen:
                    break
                newword = text[(endindex + 1): (i + 1)]
                bigram = entry.word + " " + newword
                if newword in self.Pu.keys() or len(newword) <= 3:
                    # compute conditional probability (cpr)
                    if bigram in self.Pb.keys() and entry.word in self.Pu.keys():
                        cpr = self.Pb[bigram] / self.Pu[entry.word]
                    else:
                        cpr = self.Pu(newword)
                    # apply Jelinek-Mercer Smoothing
                    newentry = Entry(newword, endindex + 1, entry.log_prob + log10(self.lam * cpr + (1 - self.lam) * self.Pu(newword)), entry)
                    if newentry not in heap:
                        heap.append(newentry)

        # get the best segmentation
        segmentation = []
        finalindex = len(text) - 1
        finalentry = chart[finalindex]
        while finalentry != None:
            segmentation.append(finalentry.word)
            finalentry = finalentry.back_ptr


        segmentation.reverse()

        return segmentation


In [12]:
Pu = Pdist(data=datafile("../data/count_1w.txt"), missingfn=avoid_long_words)
Pb = Pdist(data=datafile("../data/count_2w.txt"), missingfn=lambda x, y: 0)
lam = 0.2425
segmenter = Segment(Pu, Pb, lam) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


In [13]:
import sys
sys.path.append("..")
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

score: 0.92


### Discussion

In the third attempt, we did the following modifications:
* Set maxlen = 10, which set a bound for the maximum length of words (too long words seems to be unilkely)
* When pushing Entry to the words, we also push words with length <= 3 (with a very small probability) besides matched words. In this way, more possible segmentation are taken into account compared to length == 1 in our last attempt.
* Tuning paramter $\lambda$ for Jelinek-Mercer Smoothing. 